Link to Kaggle dataset: https://www.kaggle.com/datasets/deathtrooper/multichannel-glaucoma-benchmark-dataset/data

In [1]:
import zipfile
import pandas as pd
import keras
import os
import numpy as np
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Input, Concatenate, ReLU, AveragePooling2D, UpSampling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 
from keras.layers import GlobalMaxPool2D

In [2]:
# Unzip files for processing
with zipfile.ZipFile("full-fundus.zip","r") as zip_ref:
    zip_ref.extractall("full-fundus")
    
with zipfile.ZipFile("blood-vessel.zip","r") as zip_ref:
    zip_ref.extractall("blood-vessel")


In [3]:
# Read in metadata, and filter for a specific subset of images
# TODO: will add more variety of images later
image_data = pd.read_csv('metadata.csv')
image_data = image_data[image_data['names'].str.contains('FIVES', case=True) | image_data['names'].str.contains('HAGIS', case=True)
                       | image_data['names'].str.contains('LES-AV', case=True)]
image_data = image_data[['types', 'fundus', 'names']]

In [4]:
# There are some invalid data types for relevant columns
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 4524 to 12448
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   types   432 non-null    int64 
 1   fundus  432 non-null    object
 2   names   432 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.5+ KB
None


In [5]:
# Change datatypes to the desired values
image_data['types'] = image_data['types'].astype('string')
image_data['fundus'] = image_data['fundus'].astype('string')
image_data['names'] = image_data['names'].astype('string')
image_data['image_names'] = image_data['names'] + '.png'
print(image_data['image_names'])

4524      LES-AV-1.png
4525      LES-AV-2.png
4526      LES-AV-3.png
4527      LES-AV-4.png
4528      LES-AV-5.png
             ...      
12444    FIVES-396.png
12445    FIVES-397.png
12446    FIVES-398.png
12447    FIVES-399.png
12448    FIVES-400.png
Name: image_names, Length: 432, dtype: string


In [6]:
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 4524 to 12448
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   types        432 non-null    string
 1   fundus       432 non-null    string
 2   names        432 non-null    string
 3   image_names  432 non-null    string
dtypes: string(4)
memory usage: 16.9 KB
None


In [7]:
# Randomize data 
image_data_random = image_data.sample(frac=1, random_state=1)

# Split into healthy and glaucoma positive sets
healthy = image_data_random[image_data_random['types'] == "0"]
glaucoma = image_data_random[image_data_random['types'] == "1"]

print(f"Number of healthy samples: {len(healthy)}")
print(f"Number of unhealthy samples: {len(glaucoma)}")

Number of healthy samples: 261
Number of unhealthy samples: 171


In [8]:
# Partition data into test and train sets
healthy_train_size = 175
glaucoma_train_size = 105

healthy_test_subset = image_data_random.head(len(healthy) - healthy_train_size)
glaucoma_test_subset = image_data_random.head(len(glaucoma) - glaucoma_train_size)
test_subset = pd.concat([healthy_test_subset, glaucoma_test_subset])

healthy_train_subset = image_data_random.tail(healthy_train_size)
glaucoma_train_subset = image_data_random.tail(glaucoma_train_size)
train_subset = pd.concat([healthy_train_subset, glaucoma_train_subset])

print(f"Test size: {len(test_subset)}")
print(f"Train size: {len(train_subset)}")

Test size: 152
Train size: 280


In [9]:
train_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_train_data = train_data_generator.flow_from_dataframe(dataframe=train_subset, 
                                            batch_size= 8, 
                                            shuffle=True, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')

test_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_test_data = train_data_generator.flow_from_dataframe(dataframe=test_subset, 
                                            batch_size= 1, 
                                            shuffle=False, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')



Found 280 validated image filenames belonging to 2 classes.
Found 152 validated image filenames belonging to 2 classes.


In [10]:
def conv_model(image_size):
    
    m_input = keras.Input(shape=(image_size, image_size, 3))
    transfer = keras.applications.MobileNetV3Large(
        weights='imagenet', include_top= False, input_tensor= m_input, alpha=0.75
    )
    m_output = Dropout(0.5)(transfer.output)
    m_output = Conv2D(filters=256, kernel_size=1)(m_output) 
    m_output = GlobalMaxPool2D()(m_output)
    m_output = Dropout(0.5)(m_output)
    m_output = Dense(2, activation='softmax')(m_output)

    return keras.Model(inputs=m_input, outputs=m_output)

model = conv_model(image_size=224)

# Reduce the learning rate if an epoch occurs where there is no improvement to the output of the loss function
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=1, min_lr=1e-6)

# Using the Adam optimizer with binary cross entropy, compile the model using the given metrics
model.compile(optimizer= Adam(1e-5), 
              loss='binary_crossentropy', 
              metrics=['binary_accuracy',keras.metrics.AUC(),keras.metrics.Precision(), keras.metrics.Recall()])

trained_model = model.fit(flow_train_data, 
                    steps_per_epoch= len(flow_train_data) // 8,
                    validation_data= flow_test_data, 
                    validation_steps= len(flow_test_data), 
                    epochs=20, 
                    callbacks=[reduce_lr])

Epoch 1/20


/Users/ThomasMcBride/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 812ms/step - auc: 0.2720 - binary_accuracy: 0.3458 - loss: 5.5513 - precision: 0.3458 - recall: 0.3458 - val_auc: 0.2999 - val_binary_accuracy: 0.3553 - val_loss: 1.6820 - val_precision: 0.3553 - val_recall: 0.3553 - learning_rate: 1.0000e-05
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - auc: 0.5803 - binary_accuracy: 0.5458 - loss: 5.0855 - precision: 0.5458 - recall: 0.5458 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05
Epoch 3/20


2024-03-29 14:26:53.717459: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/ThomasMcBride/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 674ms/step - auc: 0.3656 - binary_accuracy: 0.3708 - loss: 4.6933 - precision: 0.3708 - recall: 0.3708 - val_auc: 0.3088 - val_binary_accuracy: 0.3816 - val_loss: 1.6220 - val_precision: 0.3816 - val_recall: 0.3816 - learning_rate: 8.0000e-06
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - auc: 0.4004 - binary_accuracy: 0.4333 - loss: 5.3080 - precision: 0.4333 - recall: 0.4333 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 6.4000e-06
Epoch 5/20


...

2024-03-29 14:27:15.192894: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 660ms/step - auc: 0.3804 - binary_accuracy: 0.3833 - loss: 3.7481 - precision: 0.3833 - recall: 0.3833 - val_auc: 0.3624 - val_binary_accuracy: 0.4211 - val_loss: 1.4749 - val_precision: 0.4211 - val_recall: 0.4211 - learning_rate: 1.0000e-06
Epoch 18/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - auc: 0.4036 - binary_accuracy: 0.3229 - loss: 4.5012 - precision: 0.3229 - recall: 0.3229 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-06
Epoch 19/20


2024-03-29 14:27:17.940281: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-29 14:27:17.945996: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 666ms/step - auc: 0.2190 - binary_accuracy: 0.3125 - loss: 5.4848 - precision: 0.3125 - recall: 0.3125 - val_auc: 0.3679 - val_binary_accuracy: 0.4079 - val_loss: 1.4679 - val_precision: 0.4079 - val_recall: 0.4079 - learning_rate: 1.0000e-06
Epoch 20/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step - auc: 0.3835 - binary_accuracy: 0.3625 - loss: 5.2920 - precision: 0.3625 - recall: 0.3625 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-06


2024-03-29 14:27:21.206419: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [11]:
# Demonstration:
print(trained_model)
img_path = 'full-fundus/full-fundus/OIA-ODIR-TEST-ONLINE-252.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation)  # Assuming preprocess_input is defined in your code

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
else:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
[[0.03991543 0.96008456]]
Prediction: Healthy, with 96.00845575332642% confidence
